In [ ]:
import pandas as pd
import numpy as np
from Options_Pricing import OptionsPricing

class EnrichOptionsPrices():
    
    def __init__(self):
        # constructor 
        self.op = OptionsPricing()

    def enrich_options_pricing_grid(self,
                                    underlying_value,
                                    calls_bids_values, 
                                    calls_asks_values,
                                    puts_bids_values,
                                    puts_asks_values,
                                    strike_values,
                                    pv_strike_values
                                   ):

        # Ensure array inputs
        underlying_value = float(underlying_value)
        input_columns = [np.asarray(calls_bids_values, dtype=float),
                         np.asarray(calls_asks_values, dtype=float),
                         np.asarray(puts_bids_values, dtype=float),
                         np.asarray(puts_asks_values, dtype=float),
                         np.asarray(strike_values, dtype=float),
                         np.asarray(pv_strike_values, dtype=float)
                        ]

        # Validate length consistency
        n = len(strike_values)
        if not all(len(x) == n for x in input_columns):
            raise ValueError("All input columns must have the same length.")

        # Define the three-level column index
        tuples = [('mkt_values', 'calls', 'bids'),
                  ('mkt_values', 'calls', 'asks'),
                  ('mkt_values', 'puts' , 'bids'),
                  ('mkt_values', 'puts' , 'asks'),
                  ('strike_values', 'strikes', 'market'),
                  ('strike_values', 'strikes', 'pv'),
                 ]

        # Create a MultiIndex from the tuples
        columns = pd.MultiIndex.from_tuples(tuples)

        # Create df
        df = pd.DataFrame(np.column_stack(input_columns), columns=columns)

        # Calc intrinsics
        call_intrinsic, put_intrinsic = self.op.intrinsic_value(underlying_value, 
                                                                df[('strike_values', 'strikes', 'pv')])
        df[('intrinsic_values', 'calls', "intrinsics")] = call_intrinsic
        df[('intrinsic_values', 'puts' , "intrinsics")] = put_intrinsic

        # Calc time_values
        for put_call in ['puts', 'calls']:
            for bid_ask in ['bids', 'asks']:
                df[('time_values', put_call, bid_ask)] = (df[('mkt_values', put_call, bid_ask)] - 
                                                          df[('intrinsic_values', put_call, 'intrinsics')])

        # Prepare to find "best" enr_time_values for each strike
        strikes    = df[('strike_values', 'strikes', 'market')].values  
        below_mask = strikes <= underlying_value
        above_mask = strikes > underlying_value

        # Find the highest (i.e., maximum) time values for the bid price of each strike
        # First, find the maximum time value for the bid side by strike - treat nan's as zero
        df[('enr_time_values', 'puts_and_calls', 'bids')] = np.maximum(0,
                                                                       np.nan_to_num(df[('time_values', 'calls', 'bids')], nan=0.0),
                                                                       np.nan_to_num(df[('time_values', 'puts' , 'bids')], nan=0.0))

        # Second, for strikes below the underlying, find the expanding maximum
        below_bids = df.loc[below_mask, ('enr_time_values', 'puts_and_calls', 'bids')].to_numpy()
        df.loc[below_mask, ('enr_time_values', 'puts_and_calls', 'bids')] = np.maximum.accumulate(below_bids])

        # Third, and final, for strikes above the underlying, find the expanding maximum but in reverse
        above_bids = df.loc[above_mask, ('enr_time_values', 'puts_and_calls', 'bids')].to_numpy()
        df.loc[above_mask, ('enr_time_values', 'puts_and_calls', 'bids')] = np.maximum.accumulate(above_bids[::-1])[::-1]        
        
        # Repeat proces for asks but find the lowest (i.e., minimum) time values instead      
        # First, find the minimum time value for the ask side by strike - ignore nan's (cannot treat nan's as zero)        
        df[('enr_time_values', 'puts_and_calls', 'asks')] = np.fmin(df[('time_values', 'calls', 'asks')],
                                                                    df[('time_values', 'puts' , 'asks')]) 
        
### See block below for large dataset implementation      
        # Second, for strikes above the underlying, find the expanding minimum
        above_asks = df.loc[above_mask, ('enr_time_values', 'puts_and_calls', 'asks')].to_numpy()
        df.loc[above_mask, ('enr_time_values', 'puts_and_calls', 'asks')] = \
                                                pd.Series(above_asks).expanding().min(skipna=True).to_numpy()
        
        # Third, and final, for strikes below the underlying, find the expanding maximum but in reverse
        below_asks = df.loc[below_mask, ('enr_time_values', 'puts_and_calls', 'asks')].to_numpy()
        df.loc[below_mask, ('enr_time_values', 'puts_and_calls', 'asks')] = \
                                                pd.Series(below_asks[::-1]).expanding().min(skipna=True).to_numpy()[::-1]
### End of large dataset implementation replacement
        
        # Calc enr_prices from enr_time_values and intrinsic_values
        for put_call in ['puts', 'calls']:
            for bid_ask in ['bids', 'asks']:
                df[('enr_values', put_call, bid_ask)] = (df[('intrinsic_values', put_call, "intrinsics")] +
                                                         df[('enr_time_values', 'puts_and_calls', bid_ask)])
                
        return df


In [ ]:
### Consider this implementation for larger datasets > 1000 lines instead of code in between ###

        def cummin_ignore_nan(arr):
            """Cumulative minimum ignoring NaNs."""
            result = np.empty_like(arr, dtype=float)
            running_min = np.inf
            for i, x in enumerate(arr):
                if not np.isnan(x):
                    running_min = min(running_min, x)
                result[i] = running_min if running_min != np.inf else np.nan
            return result

        # Above underlying: forward cumulative min
        df.loc[above_mask, ('enr_time_values', 'puts_and_calls', 'asks')] = cummin_ignore_nan(asks[above_mask])
        # Below underlying: reversed cumulative min
        df.loc[below_mask, ('enr_time_values', 'puts_and_calls', 'asks')] = cummin_ignore_nan(asks[below_mask][::-1])[::-1]
